### Create file with Num of poses for each exercise

In [4]:
import os
import pandas as pd

folder_path = "Exercises"

# Dictionary to store exercise names and pose counts
exercise_dict = {}

for filename in os.listdir(folder_path):
        
        parts = filename.split()
        exercise_name = " ".join(parts[:-1])
        pose_number = parts[-1]
        exercise_key = f"{exercise_name}"

        if exercise_key in exercise_dict:
            exercise_dict[exercise_key] += 1
        else:
            exercise_dict[exercise_key] = 1

df = pd.DataFrame(list(exercise_dict.items()), columns=["Exercise Name", "Poses Count"])

excel_filename = "Exercises//Pose Counts.xlsx"
df.to_excel(excel_filename, index=False)

print(f"Excel file '{excel_filename}' created successfully.")


Excel file 'Exercises//Pose Counts.xlsx' created successfully.


### Calculate Main Angles

In [ ]:
def Calculate_Main_Angles( image_path ) :
    
    image = cv2.imread(image_path)
    height, width, _ = image.shape
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = Pose.process(image)
    
    if results.pose_landmarks:
        L_Marks = results.pose_landmarks.landmark


    Angles = []
    for Key, Val in Main_Angles.items():

        Angle = int(Calculate_Angle([L_Marks[Val[0]].x, L_Marks[Val[0]].y],
                                    [L_Marks[Val[1]].x, L_Marks[Val[1]].y],
                                    [L_Marks[Val[2]].x, L_Marks[Val[2]].y]))
        Angles.append(Angle)

    return Angles


### Create file with exercises' extracted Angles

In [ ]:
def Extract_Exercises_Features(folder_path):
    
    df = pd.read_excel("Data//Pose Counts.xlsx")

    # A list to store Exercise names and angles for each pose
    exercise_angles_list = []

    for index, row in df.iterrows():

        exercise_name = row['Exercise Name']
        pose_count = row['Poses Count']
        # Calculate main angles for each pose
        pose_angles = [Calculate_Main_Angles(os.path.join(folder_path, f"{exercise_name} {pose}.png")) for pose in range(1, pose_count + 1)]
        exercise_angles_list.append([exercise_name, pose_angles])

    df_result = pd.DataFrame(exercise_angles_list, columns=["Exercise Name", "Pose Angles"])

    df_result.to_excel("Data//Exercises Features.xlsx", index=False)

    print("Excel file created successfully.")

Extract_Exercises_Features("Exercises")